In [587]:
import math
import random

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def calculate_gradient(output_i, delta_j):
        return output_i * delta_j

def calculate_weight_delta(learning_rate, delta_j, bias=False, output_i=None):
    if bias:
        return learning_rate * delta_j
    else:
        return learning_rate * calculate_gradient(output_i, delta_j)

In [588]:
class Neuron():
    def __init__(self, name, weights, bias, output_neuron=False):
        self.name = name
        self.weights = [float(x) for x in weights]
        self.bias = float(bias)
        self.delta = None
        self.weight_deltas = weights
        self.bias_delta = None
        self.output_neuron = output_neuron
        self.previous_input = None

    def __str__(self):
        return f"Neuron '{self.name}' with input weights {self.weights}, bias {self.bias} and error {self.delta}."
    
    def output(self, input):
        self.previous_input = input
        if type(input) == int:
            input = [input]
        if len(input) != len(self.weights):
            raise ValueError("The number of inputs must be equal to the number of weights.")
        total = 0.0
        for i in range(len(input)):
            total += input[i] * self.weights[i]
        return sigmoid(total + self.bias)
    
    def calculate_deltas(self, input, learning_rate):
        for i in range(len(self.weights)):
            self.weight_deltas[i] = self.weights[i] - calculate_weight_delta(learning_rate, self.delta, output_i=input[i])
        self.bias_delta = self.bias - calculate_weight_delta(learning_rate, self.delta, bias=True)
    
    def update_error(self, input, desired=None, weights=None, deltas=None):
        output = self.output(input)
        if self.output_neuron:
            self.delta = output * (1 - output) * - (desired - output)
        else:
            self.delta = output * (1 - output) * sum([weight * delta for weight, delta in zip(weights, deltas)])
        
class NeuronLayer():
    def __init__(self, name, neurons):
        self.name = name
        self.neurons = neurons

    def __str__(self):
        return f"NeuronLayer '{self.name}' with {len(self.neurons)} neurons."

    def output(self, input):
        return [p.output(input) for p in self.neurons] if hasattr(self.neurons, '__iter__') else self.perceptrons.output(input)
        
class NeuronNetwork():
    def __init__(self, name, layers):
        self.name = name
        self.layers = layers

    def __str__(self):
        return f"NeuronNetwork '{self.name}' with {len(self.layers)} layers."

    def feed_forward(self, input):
        for layer in self.layers:
            # Feed forward
            input = layer.output(input)
        return input
    
    def back_propagate(self, activation, desired, learning_rate):
        # Ik loop achterstevoren door het netwerk heen, dus ik begin bij de outputlayer
        for i in range(len(self.layers) - 1, -1, -1):
            for j in range(len(self.layers[i].neurons) - 1, -1, -1):
                cur_neuron = self.layers[i].neurons[j]
                # Eerst de output neurons, ik check voor de zekerheid of het een output neuron is en of het de laatste laag is
                if cur_neuron.output_neuron == True and i == len(self.layers) - 1:
                    output_neuron = cur_neuron
                    output_neuron.update_error(output_neuron.previous_input, desired)
                    output_neuron.calculate_deltas(output_neuron.previous_input, learning_rate)
                # Nu voor de hidden layers
                else:
                    weights = []
                    deltas = []
                    for neuron in self.layers[i + 1].neurons:
                        weights.append(neuron.weights[j])
                        deltas.append(neuron.delta)
                    cur_neuron.update_error(cur_neuron.previous_input, weights=weights, deltas=deltas)
                    cur_neuron.calculate_deltas(cur_neuron.previous_input, learning_rate)
        
    def update(self):
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.weights = neuron.weight_deltas
                neuron.bias = neuron.bias_delta

    def mean_squared_error(self, output, target):
        return (target - output) ** 2
        

    def train(self, inputs, targets, learning_rate, epochs):
        for i in range(epochs):
            total_loss = 0
            for j in range(len(inputs)):
                print_network(self)
                output = self.feed_forward(inputs[j])[0]
                loss = self.mean_squared_error(output, targets[j])
                total_loss += loss
                self.back_propagate(output, targets[j], learning_rate)
                self.update()
            average_loss = total_loss / len(inputs)
            print(f"Epoch {i + 1} - Loss: {average_loss}")

def print_network(network):
    print(f"Network '{network.name}'")
    for layer in network.layers:
        print(f"Layer '{layer.name}'")
        for neuron in layer.neurons:
            print(neuron)
        print()


In [589]:
# Random weights
weights_f = [random.uniform(-2, 2) for i in range(2)]
weights_g = [random.uniform(-2, 2) for i in range(2)]
weights_o = [random.uniform(-2, 2) for i in range(2)]



# Hidden layer
f = Neuron("F", weights_f, 0.0)
g = Neuron("G", weights_g, 0.0)

hiddenlayer = NeuronLayer("Hidden", [f, g])

# Output layer
o = Neuron("O", weights_o, 0.0, output_neuron=True)

outputlayer = NeuronLayer("Output", [o])

xor = NeuronNetwork("XOR", [hiddenlayer, outputlayer])

In [590]:
XOR_train = [
    ((0, 0), 0),
    ((0, 1), 1),
    ((1, 0), 1),
    ((1, 1), 0)
]

inputs = [left for left, right in XOR_train]
targets = [right for left, right in XOR_train]

xor.train(inputs, targets, 0.01, 10)

Network 'XOR'
Layer 'Hidden'
Neuron 'F' with input weights [1.0322768438430328, 1.144110346856814], bias 0.0 and error None.
Neuron 'G' with input weights [1.6069289655622132, -0.7469948744972097], bias 0.0 and error None.

Layer 'Output'
Neuron 'O' with input weights [1.5389632118487389, -0.3564163920692365], bias 0.0 and error None.

Network 'XOR'
Layer 'Hidden'
Neuron 'F' with input weights [1.0322768438430328, 1.144110346856814], bias -0.0005679963016412816 and error 0.05679963016412815.
Neuron 'G' with input weights [1.6069289655622132, -0.7469948744972097], bias 0.0001315451798854 and error -0.01315451798854.

Layer 'Output'
Neuron 'O' with input weights [1.538225057359706, -0.35715454655826934], bias -0.0014763089780657046 and error 0.14763089780657046.

Network 'XOR'
Layer 'Hidden'
Neuron 'F' with input weights [1.0322768438430328, 1.14425067797273], bias -0.00042766518572525704 and error -0.014033111591602455.
Neuron 'G' with input weights [1.6069289655622132, -0.7470336307641

In [591]:
print(xor.feed_forward([0, 0])[0] > 0.5)
print(xor.feed_forward([0, 1])[0] > 0.5)
print(xor.feed_forward([1, 0])[0] > 0.5)
print(xor.feed_forward([1, 1])[0] > 0.5)

True
True
True
True
